In [2]:
!pip install tensorflow_datasets

     |████████████████████████████████| 4.7 MB 11.3 MB/s            
     |████████████████████████████████| 409 kB 8.2 MB/s            
     |████████████████████████████████| 124 kB 10.9 MB/s            
     |████████████████████████████████| 52 kB 1.1 MB/s             
     |████████████████████████████████| 110 kB 11.1 MB/s            
     |████████████████████████████████| 15.7 MB 12.2 MB/s            
     |████████████████████████████████| 140 kB 11.1 MB/s            
     |████████████████████████████████| 62 kB 2.1 MB/s             
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached promise-2.3.tar.gz (19 kB)
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 155 kB 11.4 MB/s            
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
  Using cached urllib3-1.26.13-py2.py3-none-any.whl (140 kB)
     |████████████████████████████████| 1.0 MB 10.9 MB/s            
     |████████████████████████████████| 217 kB 

In [4]:
!pip install tensorflow

     |████████████████████████████████| 588.3 MB 14 kB/s              
     |████████████████████████████████| 75 kB 4.6 MB/s             
     |████████████████████████████████| 1.7 MB 10.9 MB/s            
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     |████████████████████████████████| 4.8 MB 10.9 MB/s            
     |████████████████████████████████| 14.1 MB 12.2 MB/s            
     |████████████████████████████████| 6.0 MB 10.8 MB/s            
     |████████████████████████████████| 1.1 MB 11.1 MB/s            
     |████████████████████████████████| 439 kB 10.8 MB/s            
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     |████████████████████████████████| 2.4 MB 11.1 MB/s            
     |████████████████████████████████| 4.1 MB 11.1 MB/s            
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl 

In [26]:
!pip install Pillow

     |████████████████████████████████| 3.2 MB 11.0 MB/s            
You should consider upgrading via the '/home/deniz/PycharmProjects/mobilenet_finetune_exo/venv/bin/python -m pip install --upgrade pip' command.


In [159]:
tf.config.run_functions_eagerly(True)

In [27]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [178]:
list_ds = tf.data.Dataset.list_files('./spectrogram_data_color/*.png', shuffle=False)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [60]:
for f in val_ds.take(5):
  print(f.numpy())

b'./spectrogram_data_color/7003_0_1_img.png'
b'./spectrogram_data_color/5099_0_1_img.png'
b'./spectrogram_data_color/12183_2_1_img.png'
b'./spectrogram_data_color/7752_0_1_img.png'
b'./spectrogram_data_color/19032_0_1_img.png'


In [42]:
aaa = './spectrogram_data_color/0_0_1_img.png'

In [48]:
aaa.split('/')[2].split('_')[1]

'0'

In [96]:
tf.strings.split(tf.strings.split(aaa, os.path.sep)[-1], '_')[1]

<tf.Tensor: shape=(), dtype=string, numpy=b'0'>

In [37]:
class_names = ['Level Walking', 'Ramp Descent', 'Ramp Ascent', 'Stair Descent', 'Stair Ascent']
print(class_names)

['Level Walking', 'Ramp Descent', 'Ramp Ascent', 'Stair Descent', 'Stair Ascent']


In [51]:
image_count = tf.data.experimental.cardinality(list_ds).numpy()

In [179]:
val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)

In [168]:
print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())

16585
4146


In [145]:
tf.strings.to_number(get_label(aaa), out_type=tf.dtypes.int64)

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [147]:
def get_label(file_path):
  return tf.strings.to_number(tf.strings.split(tf.strings.split(file_path, os.path.sep)[-1], '_')[1], out_type=tf.dtypes.int64)

In [148]:
def decode_img(img):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  return img

In [149]:
def process_path(file_path):
  label = get_label(file_path)
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [67]:
process_path(b'./spectrogram_data_color/19032_0_1_img.png')

TypeError: a bytes-like object is required, not 'str'

In [180]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_ds = train_ds.map(process_path, num_parallel_calls=16)
val_ds = val_ds.map(process_path, num_parallel_calls=16)

In [181]:
for image, label in train_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (224, 224, 3)
Label:  1


In [182]:
val_ds = val_ds.batch(batch_size=32)

In [183]:
train_ds = train_ds.batch(batch_size=32)

In [191]:
for image_batch, label_batch in train_ds.take(1):
   pass

image_batch.shape

TensorShape([32, 224, 224, 3])

In [19]:
import tensorflow as tf

In [193]:
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [224]:
base_model = tf.keras.applications.MobileNetV3Small(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

In [225]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 7, 7, 576)


In [226]:
base_model.trainable = False


In [227]:
base_model.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_2 (Rescaling)        (None, 224, 224, 3)  0           ['input_10[0][0]']               
                                                                                                  
 Conv (Conv2D)                  (None, 112, 112, 16  432         ['rescaling_2[0][0]']            
                                )                                                                 
                                                                                   

In [230]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')


In [231]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

base_learning_rate = 1e-5
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [232]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Functiona  (None, 7, 7, 576)        939120    
 l)                                                              
                                                                 
 global_average_pooling2d_7   (None, 576)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_7 (Dense)             (None, 1)                 577       
                                                                 
Total params: 939,697
Trainable params: 577
Non-trainable params: 939,120
_________________________________________________________________


In [233]:
validation_steps=20
initial_epochs = 200

loss0,accuracy0 = model.evaluate(val_ds, steps = validation_steps)

  2/200 [..............................] - ETA: 33s - loss: 0.5413 - accuracy: 0.3594

/home/deniz/PycharmProjects/mobilenet_finetune_exo/venv/lib/python3.7/site-packages/keras/backend.py:5677: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits, "Sigmoid", "binary_crossentropy"


130/200 [==================>...........] - ETA: 12s - loss: 0.5643 - accuracy: 0.3032WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 200 batches). You may need to use the repeat() function when building your dataset.


200/200 [==============================] - 23s 114ms/step - loss: 0.5643 - accuracy: 0.3032


In [222]:
validation_steps=10
initial_epochs = 10

loss0,accuracy0 = model.evaluate(val_ds, steps = validation_steps)

10/10 [==============================] - 2s 179ms/step - loss: 0.6931 - accuracy: 0.5312


In [201]:
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds)

Epoch 1/20
519/519 [==============================] - 632s 1s/step - loss: -0.2395 - accuracy: 0.1700 - val_loss: -0.6657 - val_accuracy: 0.1710
Epoch 2/20
519/519 [==============================] - 632s 1s/step - loss: -1.2277 - accuracy: 0.1528 - val_loss: -1.4597 - val_accuracy: 0.1712
Epoch 3/20
519/519 [==============================] - 648s 1s/step - loss: -2.1136 - accuracy: 0.1530 - val_loss: -2.2480 - val_accuracy: 0.1715
Epoch 4/20
  5/519 [..............................] - ETA: 10:01 - loss: -2.0745 - accuracy: 0.1813

KeyboardInterrupt: 

In [223]:
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds)


Epoch 1/10
519/519 [==============================] - 261s 503ms/step - loss: 0.6808 - accuracy: 0.5220 - val_loss: 0.6695 - val_accuracy: 0.5125
Epoch 2/10
519/519 [==============================] - 261s 503ms/step - loss: 0.6573 - accuracy: 0.5220 - val_loss: 0.6470 - val_accuracy: 0.5125
Epoch 3/10
519/519 [==============================] - 262s 506ms/step - loss: 0.6345 - accuracy: 0.5220 - val_loss: 0.6252 - val_accuracy: 0.5125
Epoch 4/10
369/519 [====================>.........] - ETA: 1:09 - loss: 0.6149 - accuracy: 0.5202

KeyboardInterrupt: 

In [234]:
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds)


Epoch 1/200
519/519 [==============================] - 264s 509ms/step - loss: 0.4068 - accuracy: 0.1877 - val_loss: 0.2904 - val_accuracy: 0.1792
Epoch 2/200
519/519 [==============================] - 263s 507ms/step - loss: 0.1885 - accuracy: 0.1559 - val_loss: 0.1324 - val_accuracy: 0.1717
Epoch 3/200
519/519 [==============================] - 259s 499ms/step - loss: 0.0543 - accuracy: 0.1533 - val_loss: 0.0324 - val_accuracy: 0.1712
Epoch 4/200
519/519 [==============================] - 261s 503ms/step - loss: -0.0370 - accuracy: 0.1531 - val_loss: -0.0396 - val_accuracy: 0.1712
Epoch 5/200
519/519 [==============================] - 259s 500ms/step - loss: -0.1069 - accuracy: 0.1530 - val_loss: -0.0976 - val_accuracy: 0.1712
Epoch 6/200
519/519 [==============================] - 263s 506ms/step - loss: -0.1657 - accuracy: 0.1531 - val_loss: -0.1480 - val_accuracy: 0.1712
Epoch 7/200
390/519 [=====================>........] - ETA: 59s - loss: -0.2090 - accuracy: 0.1524 

KeyboardInterrupt: 

In [ ]:
model.save('./saved_model')


In [235]:
import pickle

In [237]:
with open('../Human-Activity-Classification/Heuristic/checkpoints/CNN/CNN_bilateral_imu_emg_gon_BAND10_HOP10.pkl', 'rb') as input:
    BIO_train = pickle.load(input)


FileNotFoundError: [Errno 2] No such file or directory: '../Human-Activity-Classification/checkpoints/CNN/CNN_bilateral_imu_emg_gon_BAND10_HOP10.pkl'